# Sprint10 DeepNeuralNetwork

In [34]:
import numpy as np

# 【問題1】全結合層のクラス化
全結合層のクラス化を行なってください。


以下に雛形を載せました。コンストラクタで重みやバイアスの初期化をして、あとはフォワードとバックワードのメソッドを用意します。重みW、バイアスB、およびフォワード時の入力Xをインスタンス変数として保持しておくことで、煩雑な入出力は不要になります。


なお、インスタンスも引数として渡すことができます。そのため、初期化方法のインスタンスinitializerをコンストラクタで受け取れば、それにより初期化が行われます。渡すインスタンスを変えれば、初期化方法が変えられます。


また、引数として自身のインスタンスselfを渡すこともできます。これを利用してself.optimizer.update(self)という風に層の重みの更新が可能です。更新に必要な値は複数ありますが、すべて全結合層が持つインスタンス変数にすることができます。


初期化方法と最適化手法のクラスについては後述します。

In [35]:
class FC():
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, input, output, initializer, optimizer):
        self.optimizer = optimizer

        # initializerのメソッドを使い、self.Wとself.Bを初期化する       
        self.W = initializer.W(input, output)
        self.B = initializer.B(output)


    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.X = X

        A = np.dot(X, self.W ) + self.B

        return A



    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        dZ = np.dot(dA, self.W.T)
        self.dB = np.sum(dA, axis=0)
        self.dW = np.dot(self.X.T, dA)

        # 更新
        self = self.optimizer.update(self)

        return dZ

# 【問題2】初期化方法のクラス化
初期化を行うコードをクラス化してください。


前述のように、全結合層のコンストラクタに初期化方法のインスタンスを渡せるようにします。以下の雛形に必要なコードを書き加えていってください。標準偏差の値（sigma）はコンストラクタで受け取るようにすることで、全結合層のクラス内にこの値（sigma）を渡さなくてすむようになります。


これまで扱ってきた初期化方法はSimpleInitializerクラスと名付けることにします。

In [36]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
            個別の重み
        """
        W = np.random.randn(n_nodes1, n_nodes2) * self.sigma

        return W

    def B(self, n_nodes1):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
            個別のバイアス
        """
        B = np.random.randn(1, n_nodes1) * self.sigma

        return B

# 【問題3】最適化手法のクラス化
最適化手法のクラス化を行なってください。


最適化手法に関しても初期化方法同様に全結合層にインスタンスとして渡します。バックワードのときにself.optimizer.update(self)のように更新できるようにします。以下の雛形に必要なコードを書き加えていってください。


これまで扱ってきた最適化手法はSGDクラス（Stochastic Gradient Descent、確率的勾配降下法）として作成します。

In [37]:
class SGD():
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB

# 【問題4】活性化関数のクラス化
活性化関数のクラス化を行なってください。


ソフトマックス関数のバックプロパゲーションには交差エントロピー誤差の計算も含む実装を行うことで計算が簡略化されます。

In [38]:
class Softmax:
    '''
    ソフトマックス関数の計算を行う
    '''
    def forward(self, A):
        self.Z = np.exp(A) / np.sum(np.exp(A), axis=1).reshape(-1, 1)
        return  self.Z

    def backward(self, Y):
        #ロス計算（クロスエントロピー）
        delta = 1e-7
        loss = (-1) * np.sum( Y * np.log(self.Z) + delta ) / len(Y)

        return self.Z - Y,  loss


class Tanh:
    '''
    ハイパボリックタンジェント関数の計算を行う
    '''
    def forward(self, A):
        self.A = A
        
        return np.tanh(A)
    
    def backward(self, dZ):
        return dZ * (1 - np.tanh(self.A) ** 2)

# 【問題5】ReLUクラスの作成
現在一般的に使われている活性化関数であるReLU（Rectified Linear Unit）をReLUクラスとして実装してください。


ReLUは以下の数式です。

$$
f
(
x
)
=
R
e
L
U
(
x
)
=
{
x
if 
x
>
0
,
0
if 
x
≦
0
.
$$

$x$ : ある特徴量。スカラー


実装上はnp.maximumを使い配列に対してまとめて計算が可能です。


numpy.maximum — NumPy v1.15 Manual


一方、バックプロパゲーションのための $x$ に関する $f(x)$ の微分は以下のようになります。

$$
∂
f
(
x
)
∂
x
=
{
1
if 
x
>
0
,
0
if 
x
≦
0
.
$$

数学的には微分可能ではないですが、 $x=0$ のとき $0$ とすることで対応しています。


フォワード時の $x$ の正負により、勾配を逆伝播するかどうかが決まるということになります。

In [39]:
class ReLU:
    '''
    ReLu関数の計算を行う
    '''
    def __init__(self):# インスタンス変数　maskの初期化
        self.mask = None
        
    def forward(self,A):#信号の大きさをxとして引数として渡す
        self.mask = (A <=0 ) #  x <=0　の場合、True, それ以外はFalse を渡す
        out = A.copy() #  xの値（配列）をコピーする
        out[self.mask] = 0 # True の要素の値のみを０のに変換する
        
        return out
    
    def backward(self, Z):
        Z[self.mask] = 0 # x<=0がtrue のものは、逆伝播の微分のdoutも0で流す。それ以外はそのまま流す。
        dx = Z
        
        return dx

# 【問題6】重みの初期値
ここまでは重みやバイアスの初期値は単純にガウス分布で、標準偏差をハイパーパラメータとして扱ってきました。しかし、どのような値にすると良いかが知られています。シグモイド関数やハイパボリックタンジェント関数のときは Xavierの初期値 （またはGlorotの初期値）、ReLUのときは Heの初期値 が使われます。


XavierInitializerクラスと、HeInitializerクラスを作成してください。

In [40]:
class XavierInitializer:
    '''
    シグモイド関数・ハイパボリックタンジェント関数を使用する際の重みの初期値
    '''
    def W(self, node_input, node_output):
        self.sigma = np.sqrt(1 / node_input)
        W = self.sigma * np.random.randn(node_input, node_output)

        return W

    def B(self, node_output):
        B = self.sigma * np.random.randn(node_output)

        return B
    

class HeInitializer:
    '''
    ReLU関数を使用する際の重みの初期値
    '''
    def W(self, node_input, node_output):
        self.sigma = np.sqrt(2 / node_input)
        W = self.sigma * np.random.randn(node_input, node_output)

        return W

    def B(self, node_output):
        B = self.sigma * np.random.randn(node_output)

        return B

# 【問題7】最適化手法
学習率は学習過程で変化させていく方法が一般的です。基本的な手法である AdaGrad のクラスを作成してください。


まず、これまで使ってきたSGDを確認します。

$$
W
′
i
=
W
i
−
α
E
(
∂
L
∂
W
i
)
B
′
i
=
B
i
−
α
E
(
∂
L
∂
B
i
)
$$

$\alpha$ : 学習率（層ごとに変えることも可能だが、基本的にはすべて同じとする）


$\frac{\partial L}{\partial W_i}$ : $W_i$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial B_i}$ : $B_i$ に関する損失 $L$ の勾配


$E()$ : ミニバッチ方向にベクトルの平均を計算


続いて、AdaGradです。バイアスの数式は省略しますが、重みと同様のことをします。


更新された分だけその重みに対する学習率を徐々に下げていきます。イテレーションごとの勾配の二乗和 $H$ を保存しておき、その分だけ学習率を小さくします。


学習率は重み一つひとつに対して異なることになります。

$$
H
′
i
=
H
i
+
E
(
∂
L
∂
W
i
)
×
E
(
∂
L
∂
W
i
)
W
′
i
=
W
i
−
α
1
√
H
′
i
E
(
∂
L
∂
W
i
)
$$

$H_i$ : i層目に関して、前のイテレーションまでの勾配の二乗和（初期値は0）


$H_i^{\prime}$ : 更新した $H_i$

In [45]:
class Adagrad:
    '''
    学習率の最適化
    '''
    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1
        
    def update(self, layer):
        self.HW += layer.dW ** 2
        self.HB += layer.dB ** 2

        layer.W -= self.lr * np.sqrt(1/self.HW) * layer.dW
        layer.B -= self.lr * np.sqrt(1/self.HB) * layer.dB

# 【問題8】クラスの完成
任意の構成で学習と推定が行えるScratchDeepNeuralNetrowkClassifierクラスを完成させてください。

In [42]:
from keras.datasets import mnist
import numpy as np

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)


from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_val[:, np.newaxis])

In [43]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [46]:
class ScratchDeepNeuralNetrowkClassifier():
    '''
    ベースのクラス
    ---

    '''
    def __init__(self, sigma, lr, n_node1, n_node2, n_output, n_epoch, batch_size, activation, optimizer, verbose=False):
        self.verbose = verbose
        self.sigma = sigma
        self.lr = lr
        self.n_nodes1 = n_node1
        self.n_nodes2 = n_node2
        self.n_output = n_output
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.activation = activation
        self.optimizer = optimizer

        #activationによってW・Bの初期化を変える（elseはReLU想定）
        if activation == Tanh or activation == Sigmoid:
            self.Initializer = XavierInitializer
        else:
            self.Initializer = HeInitializer

        self.loss_train = []
        self.loss_val = []
    

    def fit(self, X, y, X_val=None, y_val=None):
        '''
        NN分類器を学習する
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        '''
        self.n_features = X.shape[1]



        #初期化（インスタンス化）
        self.FC1 = FC(self.n_features, self.n_nodes1, SimpleInitializer(self.sigma), self.optimizer(self.lr) )
        self.activation1 = self.activation()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), self.optimizer(self.lr) )
        self.activation2 = self.activation()
        self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), self.optimizer(self.lr) )
        self.activation3 = Softmax()


        
        #エポックループ
        for epoch in range(n_epoch):

            #ミニバッチ取得
            get_mini_batch = GetMiniBatch(X, y, batch_size=batch_size)

            #バッチループ
            for mini_X_train, mini_y_train in get_mini_batch:
                self.forward_propagation(mini_X_train)
                self.backward_propagation(mini_y_train)

            self.forward_propagation(X)
            dA3, loss = self.activation3.backward(y)
            self.loss_train.append(loss)

            if X_val is not None:
                self.forward_propagation(X_val)
                dA3, loss = self.activation3.backward(y_val)
                self.loss_val.append(loss)


            if self.verbose:
                #verboseをTrueにした際は学習過程などを出力する
                print()
        




    def forward_propagation(self, X):
        A1 = self.FC1.forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)


                

    def backward_propagation(self, y):
        dA3, _ = self.activation3.backward(y) # 交差エントロピー誤差とソフトマックスを合わせている
        dZ2 = self.FC3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.FC2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dZ0 = self.FC1.backward(dA1) # dZ0は使用しない




    def predict(self, X):
        '''
        ニューラルネットワーク分類器を使い推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
        Returns
        -------
        次の形のndarray, shape (n_samples, 1)
            推定結果
        '''

        self.forward_propagation(X)
        return np.argmax(self.activation3.__dict__['Z'], axis=1)


sigma= 0.01
lr = 0.01
n_nodes1 = 400
n_nodes2 = 200
n_output = 10
n_epoch = 5
batch_size = 20
activation = Tanh #Tanh or ReLU
optimizer = Adagrad #SGD or Adagrad

a = ScratchDeepNeuralNetrowkClassifier(sigma, lr, n_nodes1, n_nodes2, n_output, n_epoch, batch_size, activation, optimizer)
a.fit(X_train, y_train_one_hot, X_val, y_test_one_hot)

!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [47]:
from sklearn.metrics import accuracy_score
Y_pred_train = a.predict(X_train)
accuracy_score(y_train, Y_pred_train)

0.950375

In [48]:
a.loss_train

[0.29996017053747576,
 0.23789095867733734,
 0.20831471563278403,
 0.18085663235886795,
 0.16422919694948063]

# 【問題9】学習と推定
層の数や活性化関数を変えたいくつかのネットワークを作成してください。そして、MNISTのデータを学習・推定し、Accuracyを計算してください。

In [ ]:
sigma= 0.01
lr = 0.01
n_nodes1 = 400
n_nodes2 = 200
n_output = 10
n_epoch = 10
batch_size = 20
activation = ReLU #Tanh or ReLU

b = ScratchDeepNeuralNetrowkClassifier(sigma, lr, n_nodes1, n_nodes2, n_output, n_epoch, batch_size, activation)
b.fit(X_train, y_train_one_hot, X_val, y_test_one_hot)

In [ ]:
Y_pred_train = b.predict(X_train)
accuracy_score(y_train, Y_pred_train)

In [ ]:
b.loss_train